In [79]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/msbd5001fall2020/rainfall_2017.csv
/kaggle/input/msbd5001fall2020/test.csv
/kaggle/input/msbd5001fall2020/rainfall_2018.csv
/kaggle/input/msbd5001fall2020/train.csv
/kaggle/input/msbd5001fall2020/sampleSubmission.csv


In [80]:
import pandas as pd
train_df = pd.read_csv("/kaggle/input/msbd5001fall2020/train.csv", parse_dates = ['date'])
test_df = pd.read_csv("/kaggle/input/msbd5001fall2020/test.csv", parse_dates = ['date'])

In [81]:
# rainfall_2017 = pd.read_csv("/kaggle/input/msbd5001fall2020/rainfall_2017.csv")
# rainfall_2018 = pd.read_csv("/kaggle/input/msbd5001fall2020/rainfall_2018.csv")

In [82]:
# rainfall_2017 = rainfall_2017.replace('Trace',0)
# rainfall_2017 = rainfall_2017.fillna(0)
# rainfall_2018 = rainfall_2018.replace('Trace',0)
# rainfall_2018 = rainfall_2018.fillna(0)

In [83]:
# def check_rainfall(date):
#     if date.year == 2017:
#         return rainfall_2017.iat[date.day-1,date.month]
#     else:
#         return rainfall_2018.iat[date.day-1,date.month]
    


In [84]:
from datetime import date
import holidays
hk_holidays = holidays.HongKong(years=[2017,2018])

def check_holiday(date):
    if date in hk_holidays:
        return 1
    else:
        return 0

def check_weekday(date):
    if date < 5:
        return 1
    else:
        return 0

def extract_date(df,column):
    df[column+"_year"] = df[column].apply(lambda x: x.year)
    df[column+"_month"] = df[column].apply(lambda x: x.month)
    df[column+"_day"] = df[column].apply(lambda x: x.day)
    df[column+"_hour"] = df[column].apply(lambda x: x.hour)
#     df[column+"_weekdaynum"] = df[column].apply(lambda x: x.weekday())
    df[column+"_weekday"] = df[column].dt.weekday
#     df[column+"_weekdayind"] = df[column].dt.weekday.apply(check_weekday)
    df[column+"_holiday"] = df[column].apply(check_holiday)
#     df[column+"_rainfall"] = df[column].apply(check_rainfall)

In [85]:
extract_date(train_df,'date')
extract_date(test_df,'date')

In [86]:
# train_df['date_rainfall'] = train_df['date_rainfall'].astype('float64')
# test_df['date_rainfall'] = test_df['date_rainfall'].astype('float64')

In [87]:
train_df.head(100)

,id,date,speed,date_year,date_month,date_day,date_hour,date_weekday,date_holiday
0,0,2017-01-01 00:00:00,43.002930,2017,1,1,0,6,0
1,1,2017-01-01 01:00:00,46.118696,2017,1,1,1,6,0
2,2,2017-01-01 02:00:00,44.294158,2017,1,1,2,6,0
3,3,2017-01-01 03:00:00,41.067468,2017,1,1,3,6,0
4,4,2017-01-01 04:00:00,46.448653,2017,1,1,4,6,0
...,...,...,...,...,...,...,...,...,...
95,95,2017-04-01 23:00:00,46.235164,2017,4,1,23,5,0
96,96,2017-05-01 00:00:00,47.620550,2017,5,1,0,0,1
97,97,2017-05-01 01:00:00,49.338859,2017,5,1,1,0,1
98,98,2017-05-01 02:00:00,47.116882,2017,5,1,2,0,1


In [88]:
test_df.head(100)

,id,date,date_year,date_month,date_day,date_hour,date_weekday,date_holiday
0,0,2018-01-01 02:00:00,2018,1,1,2,0,1
1,1,2018-01-01 05:00:00,2018,1,1,5,0,1
2,2,2018-01-01 07:00:00,2018,1,1,7,0,1
3,3,2018-01-01 08:00:00,2018,1,1,8,0,1
4,4,2018-01-01 10:00:00,2018,1,1,10,0,1
...,...,...,...,...,...,...,...,...
95,95,2018-10-01 08:00:00,2018,10,1,8,0,1
96,96,2018-10-01 10:00:00,2018,10,1,10,0,1
97,97,2018-10-01 17:00:00,2018,10,1,17,0,1
98,98,2018-10-01 18:00:00,2018,10,1,18,0,1


In [89]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

X = train_df.drop(['id', 'date', 'speed'], axis=1)
y = train_df['speed']

test_X = test_df.drop(['id', 'date'], axis=1)

In [90]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, y)
reg.score(X,y), mean_squared_error(reg.predict(X), y)

(0.17162828027727328, 152.615277119845)

In [91]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=20, random_state=0)
regr.fit(X, y)
regr.score(X,y), mean_squared_error(regr.predict(X), y)

(0.9827200652469894, 3.1835732294509995)

In [92]:
# Current best

from xgboost import XGBRegressor

xgbr = XGBRegressor()
xgbr.fit(X, y)

xgbr.score(X,y), mean_squared_error(xgbr.predict(X), y)

(0.9392051934072398, 11.200546849555748)

In [93]:
import catboost as ctb

ctbr = ctb.CatBoostRegressor()
ctbr.fit(X, y)

ctbr.score(X,y), mean_squared_error(ctbr.predict(X), y)

Learning rate set to 0.062437
0:	learn: 12.9497359	total: 12.2ms	remaining: 12.2s
1:	learn: 12.3541319	total: 20.6ms	remaining: 10.3s
2:	learn: 11.8073638	total: 28.8ms	remaining: 9.58s
3:	learn: 11.2935192	total: 36.6ms	remaining: 9.12s
4:	learn: 10.8183354	total: 44.7ms	remaining: 8.9s
5:	learn: 10.3832786	total: 52.6ms	remaining: 8.72s
6:	learn: 10.0068048	total: 60.4ms	remaining: 8.56s
7:	learn: 9.6330309	total: 68.3ms	remaining: 8.47s
8:	learn: 9.2928872	total: 76ms	remaining: 8.37s
9:	learn: 8.9822260	total: 84.2ms	remaining: 8.33s
10:	learn: 8.6990732	total: 91.9ms	remaining: 8.26s
11:	learn: 8.4493168	total: 99.4ms	remaining: 8.18s
12:	learn: 8.2204386	total: 107ms	remaining: 8.13s
13:	learn: 8.0054941	total: 115ms	remaining: 8.07s
14:	learn: 7.8015988	total: 122ms	remaining: 8.04s
15:	learn: 7.6120913	total: 130ms	remaining: 8s
16:	learn: 7.4385130	total: 137ms	remaining: 7.94s
17:	learn: 7.2851104	total: 144ms	remaining: 7.86s
18:	learn: 7.1470551	total: 149ms	remaining: 7.68

170:	learn: 5.1076363	total: 572ms	remaining: 2.77s
171:	learn: 5.1030804	total: 576ms	remaining: 2.77s
172:	learn: 5.1008153	total: 578ms	remaining: 2.77s
173:	learn: 5.0969559	total: 581ms	remaining: 2.76s
174:	learn: 5.0941537	total: 583ms	remaining: 2.75s
175:	learn: 5.0907390	total: 586ms	remaining: 2.74s
176:	learn: 5.0873410	total: 589ms	remaining: 2.74s
177:	learn: 5.0847500	total: 591ms	remaining: 2.73s
178:	learn: 5.0827058	total: 594ms	remaining: 2.72s
179:	learn: 5.0791544	total: 596ms	remaining: 2.72s
180:	learn: 5.0775060	total: 599ms	remaining: 2.71s
181:	learn: 5.0740396	total: 602ms	remaining: 2.71s
182:	learn: 5.0719658	total: 605ms	remaining: 2.7s
183:	learn: 5.0691755	total: 608ms	remaining: 2.69s
184:	learn: 5.0652393	total: 610ms	remaining: 2.69s
185:	learn: 5.0631060	total: 613ms	remaining: 2.68s
186:	learn: 5.0593203	total: 616ms	remaining: 2.68s
187:	learn: 5.0563103	total: 618ms	remaining: 2.67s
188:	learn: 5.0519267	total: 621ms	remaining: 2.66s
189:	learn: 5

385:	learn: 4.6292548	total: 1.15s	remaining: 1.83s
386:	learn: 4.6283745	total: 1.16s	remaining: 1.83s
387:	learn: 4.6276082	total: 1.16s	remaining: 1.83s
388:	learn: 4.6240214	total: 1.16s	remaining: 1.82s
389:	learn: 4.6225693	total: 1.16s	remaining: 1.82s
390:	learn: 4.6210933	total: 1.17s	remaining: 1.82s
391:	learn: 4.6200124	total: 1.17s	remaining: 1.81s
392:	learn: 4.6187599	total: 1.17s	remaining: 1.81s
393:	learn: 4.6176144	total: 1.17s	remaining: 1.81s
394:	learn: 4.6166672	total: 1.18s	remaining: 1.8s
395:	learn: 4.6150387	total: 1.18s	remaining: 1.8s
396:	learn: 4.6140074	total: 1.18s	remaining: 1.8s
397:	learn: 4.6131683	total: 1.19s	remaining: 1.79s
398:	learn: 4.6121395	total: 1.19s	remaining: 1.79s
399:	learn: 4.6115373	total: 1.19s	remaining: 1.79s
400:	learn: 4.6097515	total: 1.19s	remaining: 1.78s
401:	learn: 4.6074694	total: 1.2s	remaining: 1.78s
402:	learn: 4.6065720	total: 1.2s	remaining: 1.78s
403:	learn: 4.6058094	total: 1.2s	remaining: 1.77s
404:	learn: 4.6051

592:	learn: 4.3607748	total: 1.73s	remaining: 1.19s
593:	learn: 4.3600634	total: 1.73s	remaining: 1.18s
594:	learn: 4.3593802	total: 1.74s	remaining: 1.18s
595:	learn: 4.3566644	total: 1.74s	remaining: 1.18s
596:	learn: 4.3559958	total: 1.74s	remaining: 1.18s
597:	learn: 4.3553584	total: 1.74s	remaining: 1.17s
598:	learn: 4.3546896	total: 1.75s	remaining: 1.17s
599:	learn: 4.3543820	total: 1.75s	remaining: 1.17s
600:	learn: 4.3522447	total: 1.75s	remaining: 1.16s
601:	learn: 4.3515739	total: 1.76s	remaining: 1.16s
602:	learn: 4.3508501	total: 1.76s	remaining: 1.16s
603:	learn: 4.3500690	total: 1.76s	remaining: 1.16s
604:	learn: 4.3489574	total: 1.76s	remaining: 1.15s
605:	learn: 4.3470322	total: 1.77s	remaining: 1.15s
606:	learn: 4.3462353	total: 1.77s	remaining: 1.15s
607:	learn: 4.3457182	total: 1.77s	remaining: 1.14s
608:	learn: 4.3441109	total: 1.78s	remaining: 1.14s
609:	learn: 4.3428305	total: 1.78s	remaining: 1.14s
610:	learn: 4.3405588	total: 1.78s	remaining: 1.13s
611:	learn: 

770:	learn: 4.1788987	total: 2.32s	remaining: 688ms
771:	learn: 4.1782438	total: 2.32s	remaining: 685ms
772:	learn: 4.1763054	total: 2.32s	remaining: 682ms
773:	learn: 4.1755046	total: 2.33s	remaining: 679ms
774:	learn: 4.1746715	total: 2.33s	remaining: 676ms
775:	learn: 4.1739290	total: 2.33s	remaining: 673ms
776:	learn: 4.1726724	total: 2.33s	remaining: 670ms
777:	learn: 4.1719392	total: 2.34s	remaining: 667ms
778:	learn: 4.1714391	total: 2.34s	remaining: 664ms
779:	learn: 4.1709626	total: 2.34s	remaining: 661ms
780:	learn: 4.1705940	total: 2.35s	remaining: 658ms
781:	learn: 4.1692812	total: 2.35s	remaining: 655ms
782:	learn: 4.1666534	total: 2.35s	remaining: 651ms
783:	learn: 4.1661581	total: 2.35s	remaining: 648ms
784:	learn: 4.1653034	total: 2.36s	remaining: 645ms
785:	learn: 4.1650427	total: 2.36s	remaining: 642ms
786:	learn: 4.1643833	total: 2.36s	remaining: 639ms
787:	learn: 4.1636665	total: 2.36s	remaining: 636ms
788:	learn: 4.1632246	total: 2.37s	remaining: 633ms
789:	learn: 

985:	learn: 3.9874132	total: 2.9s	remaining: 41.1ms
986:	learn: 3.9869379	total: 2.9s	remaining: 38.2ms
987:	learn: 3.9863672	total: 2.9s	remaining: 35.3ms
988:	learn: 3.9861176	total: 2.9s	remaining: 32.3ms
989:	learn: 3.9854806	total: 2.91s	remaining: 29.4ms
990:	learn: 3.9850044	total: 2.91s	remaining: 26.4ms
991:	learn: 3.9834998	total: 2.91s	remaining: 23.5ms
992:	learn: 3.9830528	total: 2.92s	remaining: 20.6ms
993:	learn: 3.9822462	total: 2.92s	remaining: 17.6ms
994:	learn: 3.9817452	total: 2.92s	remaining: 14.7ms
995:	learn: 3.9813058	total: 2.92s	remaining: 11.7ms
996:	learn: 3.9808795	total: 2.92s	remaining: 8.8ms
997:	learn: 3.9801767	total: 2.93s	remaining: 5.87ms
998:	learn: 3.9798225	total: 2.93s	remaining: 2.93ms
999:	learn: 3.9793788	total: 2.93s	remaining: 0us


(0.9140476361010533, 15.835455898873818)

In [94]:
from hyperopt import hp
import numpy as np
from sklearn.metrics import mean_squared_error


# XGB parameters
xgb_reg_params = {
    'max_depth': hp.choice('max_depth', np.arange(10, 100, dtype=int)),
    'min_child_weight': hp.quniform ('min_child_weight', 1, 50, 1),
    'subsample': hp.uniform ('subsample', 0.1, 1),
    'n_estimators' : hp.choice('n_estimators', np.arange(100, 10000, 100, dtype=int)),
    'learning_rate' : hp.quniform('learning_rate', 0.025, 1.0, 0.025),
    'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.2, 1, 0.05)
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

# CTB parameters
ctb_reg_params = {
    'learning_rate' : hp.quniform('learning_rate', 0.025, 1.0, 0.025),
    'max_depth': hp.choice('max_depth', np.arange(1, 16, dtype=int)),
    'colsample_bylevel': hp.quniform('colsample_bylevel', 0.2, 1, 0.05),
    'n_estimators' : hp.choice('n_estimators', np.arange(100, 10000, 100, dtype=int)),
    'eval_metric':       'RMSE',
}
ctb_fit_params = {
    'early_stopping_rounds': 10,
    'verbose': False
}
ctb_para = dict()
ctb_para['reg_params'] = ctb_reg_params
ctb_para['fit_params'] = ctb_fit_params
ctb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

In [95]:
import xgboost as xgb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials


class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)
    
    def ctb_reg(self, para):
        reg = ctb.CatBoostRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [96]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

obj = HPOpt(X_train, X_test, y_train, y_test)

xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [01:28<00:00,  1.13trial/s, best loss: 3.8472379819045437]


In [97]:
xgb_opt

({'colsample_bytree': 0.8500000000000001,
  'gamma': 0.8500000000000001,
  'learning_rate': 0.125,
  'max_depth': 40,
  'min_child_weight': 16.0,
  'n_estimators': 93,
  'subsample': 0.9473768455265624},
 <hyperopt.base.Trials at 0x7f06ed3264d0>)

In [98]:
xgbr = XGBRegressor(**xgb_opt[0])
xgbr.fit(X, y)

xgbr.score(X,y), mean_squared_error(xgbr.predict(X), y)

(0.9697213152733647, 5.578401278506467)

In [99]:
test_df['speed'] = xgbr.predict(test_df.drop(['id', 'date'], axis=1))

In [100]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# obj = HPOpt(X_train, X_test, y_train, y_test)

# ctb_opt = obj.process(fn_name='ctb_reg', space=ctb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

In [101]:
# ctb_opt

In [102]:
# ctbr =  ctb.CatBoostRegressor(**ctb_opt[0])
# ctbr.fit(X, y)

# ctbr.score(X,y), mean_squared_error(ctbr.predict(X), y)

In [103]:
# test_df['speed'] = ctbr.predict(test_df.drop(['id', 'date'], axis=1))

In [104]:
sample_submission = test_df[['id', 'speed']]
sample_submission.to_csv(r'/kaggle/working/msbd5001fall2020_Sample_Submission.csv', index = False)